In [1]:
! python -m pip install gurobipy
! python -m pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [20]:
import pandas as pd
from gurobipy import GRB, Model, quicksum, LinExpr

# Dati
df = pd.read_csv("data/cup.csv")
idx = df.columns.tolist()[0]
df = df.set_index(f"{idx}")


# Definizione degli insiemi

## I: insieme delle località
I = ["loc1", "loc2", "loc3", "loc4", "loc5", "loc6"]

## J: insieme dei quartieri
J = ["q1", "q2", "q3", "q4", "q5", "q6"]

# Definizione dei parametri

## C: costo apertura CUP nella località i
C = {"loc1": 1, "loc2": 1, "loc3": 1, "loc4": 1, "loc5": 1, "loc6": 1}

## D: domanda di servizi CUP del quartiere j
D = {"q1": 1, "q2": 1, "q3": 1, "q4": 1, "q5": 1, "q6": 1}

## Matrice A, A_ij: tempo di spostamento dal quartiere j alla località i
A = { 
      ("q1", "loc1"): 5, ("q1", "loc2"): 10, ("q1", "loc3"): 20, ("q1", "loc4"): 30, ("q1", "loc5"): 30, ("q1", "loc6"): 20,
      ("q2", "loc1"): 10, ("q2", "loc2"): 5, ("q2", "loc3"): 25, ("q2", "loc4"): 35, ("q2", "loc5"): 20, ("q2", "loc6"): 10,
      ("q3", "loc1"): 20, ("q3", "loc2"): 25, ("q3", "loc3"): 5, ("q3", "loc4"): 15, ("q3", "loc5"): 30, ("q3", "loc6"): 20,
      ("q4", "loc1"): 30, ("q4", "loc2"): 35, ("q4", "loc3"): 15, ("q4", "loc4"): 5, ("q4", "loc5"): 15, ("q4", "loc6"): 25,
      ("q5", "loc1"): 30, ("q5", "loc2"): 20, ("q5", "loc3"): 30, ("q5", "loc4"): 15, ("q5", "loc5"): 5, ("q5", "loc6"): 14,
      ("q6", "loc1"): 20, ("q6", "loc2"): 10, ("q6", "loc3"): 20, ("q6", "loc4"): 25, ("q6", "loc5"): 14, ("q6", "loc6"): 5
    }
   

print("Località (I):", I)
print("Quartieri (J):", J)
print("Costi (C):", C)
print("Domande (D):", D)
print("Tempi di spostamento (quartiere, localita):\n", A)


# Creazione modello
m = Model("cup")

# Creazione variabili decisionali
x = m.addVars(I, vtype=GRB.BINARY, lb=0, name=[i for i in I])

# Creazione vincoli "statici" (i.e. che non dipendono dai dati) esigenze quartieri
"""
m.addConstr(x["loc1"] + x["loc2"] >= 1, name="esigenza_q1")
m.addConstr(x["loc1"] + x["loc2"] + x["loc6"] >= 1, name="esigenza_q2")
m.addConstr(x["loc3"] + x["loc4"] >= 1, name="esigenza_q3")
m.addConstr(x["loc3"] + x["loc4"] + x["loc5"] >= 1, name="esigenza_q4")
m.addConstr(x["loc4"] + x["loc5"] + x["loc6"] >= 1, name="esigenza_q5")
m.addConstr(x["loc2"] + x["loc5"] + x["loc6"] >= 1, name="esigenza_q6")
"""

# Creazione vincoli "dinamici" (i.e. che dipendono dai dati) esigenze quartieri
for j in J:
    somma = LinExpr()
    for i in I:
        tempo_spostamento = A[j,i]
        print(f"Verifica vincolo per località {i} e quartiere {j}:  tempo spostamento = {tempo_spostamento}")
        if tempo_spostamento < 15:
            somma += x[i]
    m.addConstr(somma >= 1, name=f"esigenza_dinamica_{j}")

# Creazione vincoli "dinamici" (i.e. che dipendono dai dati) esigenze quartieri con quicksum
for j in J:
    m.addConstr(quicksum(x[i] for i in I if A[j,i] < 15) >= 1, name=f"esigenza_dinamica_q{j}")


# Creazione funzione obiettivo: minimizzare i costi di apertura
m.setObjective(quicksum(C[i] * x[i] for i in I), GRB.MINIMIZE)

m.write("cup1.lp")

# Risoluzione del modello
m.optimize()

# Stampa dei risultati
if m.status == GRB.OPTIMAL:
    print("\nSoluzione ottima trovata:\n")
    for i in I:
        if x[i].x > 0.5:
            print(f"Aprire CUP nella località {i}")
    print(f"\nCosto totale di apertura: {m.objVal}")

Località (I): ['loc1', 'loc2', 'loc3', 'loc4', 'loc5', 'loc6']
Quartieri (J): ['q1', 'q2', 'q3', 'q4', 'q5', 'q6']
Costi (C): {'loc1': 1, 'loc2': 1, 'loc3': 1, 'loc4': 1, 'loc5': 1, 'loc6': 1}
Domande (D): {'q1': 1, 'q2': 1, 'q3': 1, 'q4': 1, 'q5': 1, 'q6': 1}
Tempi di spostamento (quartiere, localita):
 {('q1', 'loc1'): 5, ('q1', 'loc2'): 10, ('q1', 'loc3'): 20, ('q1', 'loc4'): 30, ('q1', 'loc5'): 30, ('q1', 'loc6'): 20, ('q2', 'loc1'): 10, ('q2', 'loc2'): 5, ('q2', 'loc3'): 25, ('q2', 'loc4'): 35, ('q2', 'loc5'): 20, ('q2', 'loc6'): 10, ('q3', 'loc1'): 20, ('q3', 'loc2'): 25, ('q3', 'loc3'): 5, ('q3', 'loc4'): 15, ('q3', 'loc5'): 30, ('q3', 'loc6'): 20, ('q4', 'loc1'): 30, ('q4', 'loc2'): 35, ('q4', 'loc3'): 15, ('q4', 'loc4'): 5, ('q4', 'loc5'): 15, ('q4', 'loc6'): 25, ('q5', 'loc1'): 30, ('q5', 'loc2'): 20, ('q5', 'loc3'): 30, ('q5', 'loc4'): 15, ('q5', 'loc5'): 5, ('q5', 'loc6'): 14, ('q6', 'loc1'): 20, ('q6', 'loc2'): 10, ('q6', 'loc3'): 20, ('q6', 'loc4'): 25, ('q6', 'loc5'): 14